In [ ]:
#!nvcc --version

In [ ]:
#!pip install turicreate skafos

In [ ]:
#!pip uninstall -y mxnet && pip install mxnet-cu100==1.3.1

# Model Training

Below we do the following:
1. Load training data from the sframe `dashlight_iomages.sframe`. This should have been created by running through the data creation pipeline notebook first.
2. Train the object detection model.
3. Save the model, testing data, and convert to coreml.
4. Evaluate the model against the test data and save the scores.
5. Upload to skafos.


In [ ]:
import os
import json
import zipfile
from datetime import datetime
import turicreate as tc
import skafos
from skafos import models
#from sagemaker import get_execution_role
#import boto3
#from utils import s3_upload, s3_download, zip_ext, tarzip_file
#s3_bucket="skafos.dashlights"
#role = get_execution_role()
#region = boto3.Session().region_name

FORMAT = "%m%d%Y%H%M%S"

In [ ]:
# Download baseline training dataset
sframe_name = "dashlight_images.sframe"

#sframe_name_zipped = sframe_name + zip_ext
#s3_download(bucket=s3_bucket, filename=sframe_name_zipped, key="datasets/sframes/" + sframe_name_zipped, unzip=True)

In [ ]:
# Load the dataset with turicreate
dataset = tc.load_sframe(sframe_name)

In [ ]:
# Create and Save training/testing data folds
train_percentage = 0.8
train_data, test_data = dataset.random_split(train_percentage)

In [ ]:
train_data.head()

In [ ]:
### MODEL TRAINING ###

# Setup GPU for training
tc.config.set_num_gpus(-1)
tc.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)

In [ ]:
# Train model
model = tc.object_detector.create(
    dataset=train_data,
    annotations="annotations",
    batch_size=32,
    max_iterations=500
)

In [ ]:
# Create new model artifacts version
version = datetime.now().strftime(FORMAT)

# Save the model here
model_name = version + "_dashlights_model"
model.save(model_name)

# Upload model to s3
#s3_upload(filename=model_name, bucket=s3_bucket, key="model-artifacts/" + VERSION + "/" + model_name)

In [ ]:
model.summary()

In [ ]:
# Write out test data
test_data.save(version + "_testing_sframe")  # sframe is good for loading and using in turi

# create a ZipFile object so we could download the test data too
zipObj = zipfile.ZipFile(version + '_test_images.zip', 'w')
 
# Add multiple files to the zip
for file in test_data['path']:
    zipObj.write(file)
    
# close the Zip File
zipObj.close()

In [ ]:
### MODEL VALIDATION ###

# Get predictions and evaluation metrics
#test_data["predictions"] = model.predict(test_data)
scores = model.evaluate(test_data, iou_threshold=0)

In [ ]:
scores

In [ ]:
# Let's get actual predictions and take a look at some model results visually
test_data['predictions'] = model.predict(test_data, iou_threshold=0)
test_data['images_w_preds'] = tc.object_detector.util.draw_bounding_boxes(test_data['image'], test_data['predictions'])

In [ ]:
# Loop through a subset of the predictions and take a look!
for i in test_data['images_w_preds'][:10]:
    i.show()

In [ ]:
# Write out the model scores
with open(version + "_model_scores.json", "w") as f:
    f.write(json.dumps(scores))

In [ ]:
# Export model to coreml
model.export_coreml("dashlights.mlmodel", iou_threshold=0)

## Export with Skafos
Use Skafos SDK to upload the model to the dashlights project.

In [ ]:
# Get account summary and api token set

os.environ["SKAFOS_API_TOKEN"] = ""
#skafos.summary()

In [ ]:
# Set our options
opts = {
    'org_name': 'skafos-demo',
    'app_name': 'DashLights',
    'model_name': 'dashlights'
}

In [ ]:
# fill in the details of this description before saving
desc = "Version ___: trained on ____ images, final loss ___, image sizes ___, <other notes>"

# Example
#Version 08202019182036: trained on ~1000 images, different icon sizes and multiple in frame, final loss 1.05, images 416x416

In [ ]:
# Upload the model
res = models.upload_version(
    files="dashlights.mlmodel",
    description=desc,
    **opts
)